In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key: str = os.getenv("api_key", "")
base_url: str = os.getenv("base_url", "")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-nano-2025-04-14",
    api_key=api_key,
    base_url=base_url,
)

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=134
)

tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.invoke({"query": "LLM"}))



Page: Large language model
Summary: A large language model (LLM) is a language model trained with self-supervised machine learning on 


In [3]:
from langchain.agents import load_tools

tools = load_tools(['wikipedia', 'llm-math'], llm=llm)
print(tools[0])
print(tools[1])

api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/amir/anaconda3/envs/myenv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)
name='Calculator' description='Useful for when you need to answer questions about math.' func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("

In [4]:
# tools[1].invoke('''2 * 3''')

In [5]:
# Напишем функцию, которая считает количество символов в слове
def word_len(word):
    return len(word)
    
word_len('langchain')

9

In [6]:
from langchain_core.tools import Tool, StructuredTool

word_len_tool = Tool(
    name = "Word length",
    func = word_len,
    description='Считает количество символов в слове'
)
print(word_len_tool)

name='Word length' description='Считает количество символов в слове' func=<function word_len at 0x72903a2c39a0>


In [7]:
word_len_tool.invoke("langchain")

9

In [11]:
def word_len_plus(word: str, b: int, g: int) -> int:
    """Считает количество символов в слове и прибавляет к ним числа""" # Обязательно добавляем docstring
    return len(word) + b - g

word_len_plus_tool = StructuredTool.from_function(func=word_len_plus)
print(word_len_plus_tool)

name='word_len_plus' description='Считает количество символов в слове и прибавляет к ним числа' args_schema=<class 'langchain_core.utils.pydantic.word_len_plus'> func=<function word_len_plus at 0x729035aa11b0>


In [13]:
word_len_plus_tool.invoke({'word': 'LLM', 'b': 2, 'g': 9})

-4

In [16]:
from langchain_core.tools import  tool

@tool
def summ(a: int, b: int)-> int:
    """Складывает 2 целых числа"""
    return a + b
print(summ.name)
print(summ.description)
print(summ.args)

summ
Складывает 2 целых числа
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [17]:
summ.invoke({'a': 3, 'b': 5})

8

In [20]:
summ.invoke({'a': 3, 'b': "5.09"})

ValidationError: 1 validation error for summ
b
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='5.09', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

In [22]:
model_with_tools = llm.bind_tools([summ])
response = model_with_tools.invoke("Сколько будет двадцать два плюс 17?")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VSKiiJ6OPCcjTgAiry2qWVHo', 'function': {'arguments': '{"a":22,"b":17}', 'name': 'summ'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 59, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CMyZ9joM9kw467xVmAxRrz0i31pBK', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--10e58936-e8dd-4056-a28c-9208a4037d0c-0', tool_calls=[{'name': 'summ', 'args': {'a': 22, 'b': 17}, 'id': 'call_VSKiiJ6OPCcjTgAiry2qWVHo', 'type': 'tool_call'}], usage_metadata={'input_tokens': 59, 'output_tokens': 18, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_r

In [23]:
print(response.tool_calls)

[{'name': 'summ', 'args': {'a': 22, 'b': 17}, 'id': 'call_VSKiiJ6OPCcjTgAiry2qWVHo', 'type': 'tool_call'}]


In [24]:
summ.invoke(response.tool_calls[0])

ToolMessage(content='39', name='summ', tool_call_id='call_VSKiiJ6OPCcjTgAiry2qWVHo')

In [25]:
# Спросим нейтральный вопрос - модель выдала ответ, tool_calls не было.
model_with_tools.invoke('Какая столица Хорватии?')

AIMessage(content='Столицей Хорватии является Загреб.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 58, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CMydVZKdV8NGe9lDae8PVGYO7JArb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c70829c0-bc2b-4972-95c7-15f56158d1cb-0', usage_metadata={'input_tokens': 58, 'output_tokens': 12, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
llm.invoke("Сколько букв в слове зачёт?").content

'В слове "зачёт" 5 букв.'

In [27]:
@tool
def get_word_length(word: str) -> int:
    """Возвращает длину слова""" # добавляем docstring, чтобмы LLM понимала суть инструмента
    return len(word)
tools = [get_word_length]

model_with_tools = llm.bind_tools(tools)

resp = model_with_tools.invoke("Сколько букв в слове зачёт?")
resp.tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'зачёт'},
  'id': 'call_QRnK7fsPjVLNIjpNAJzJeZJh',
  'type': 'tool_call'}]

In [29]:
tool_message = get_word_length.invoke(resp.tool_calls[0])
print(tool_message)

content='5' name='get_word_length' tool_call_id='call_QRnK7fsPjVLNIjpNAJzJeZJh'


In [30]:
from langchain_core.messages import HumanMessage

# теперь соберём все сообщения вместе 
# важно чтобы id в tool_calling и ToolMessage совпадали.
messages = [
    HumanMessage("Сколько букв в слове зачёт?"),
    resp,
    tool_message
]
model_with_tools.invoke(messages).content

'В слове "зачёт" 5 букв.'

In [71]:
import datetime

@tool
def get_current_time(nothing: None ) -> str:
    """Get current time""" 

    # Получаем текущее время и дату
    now = datetime.datetime.now()
    
    # Возвращаем текущее время
    return now.strftime('%H:%M:%S')




In [61]:
# Загрузим готовые инструменты
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
llm_with_tools = llm.bind_tools(tools + [get_current_time])

In [62]:
task = "Извлеки квадратный корень из девяти и назови количество участников группы Beatles. А еще скажи время"

messages = [HumanMessage(task)]

response = llm_with_tools.invoke(messages)
messages.append(response)
response.tool_calls

[{'name': 'Calculator',
  'args': {'__arg1': 'sqrt(9)'},
  'id': 'call_T45angwB4455fZMZvKE9OeK7',
  'type': 'tool_call'},
 {'name': 'wikipedia',
  'args': {'query': 'The Beatles'},
  'id': 'call_sK2FETl0X0rJggR4cTTr0M8z',
  'type': 'tool_call'}]

In [63]:
messages

[HumanMessage(content='Извлеки квадратный корень из девяти и назови количество участников группы Beatles. А еще скажи время', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_T45angwB4455fZMZvKE9OeK7', 'function': {'arguments': '{"__arg1": "sqrt(9)"}', 'name': 'Calculator'}, 'type': 'function'}, {'id': 'call_sK2FETl0X0rJggR4cTTr0M8z', 'function': {'arguments': '{"query": "The Beatles"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 159, 'total_tokens': 209, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CMz2v56rMMjlR8cR0bCxgPaXqMbor', 'service_tier': 'default', 'finish_reason': 

In [66]:
# Соберём ToolMessages
for tool_call in response.tool_calls:

    if tool_call["name"] == 'wikipedia':
        tool_msg = tools[0].invoke(tool_call)
        
    elif tool_call["name"] == 'get_current_time':
        tool_msg = get_current_time.invoke(tool_call)
        
    else:
        tool_msg = tools[1].invoke(tool_call)
        
    messages.append(tool_msg)

print(llm_with_tools.invoke(messages).content)

Квадратный корень из девяти равен 3. Количество участников группы Beatles — 4. Время, сейчас 15:02.


In [72]:
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent


url = "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
df = pd.read_csv(url)

agent = create_pandas_dataframe_agent(
    llm,
    df,
    tools = [get_current_time],
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

agent.invoke("how many rows are there? And what is current time?")

/home/amir/anaconda3/envs/myenv/lib/python3.10/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'tools': [StructuredTool(name='get_current_time', description='Get current time', args_schema=<class 'langchain_core.utils.pydantic.get_current_time'>, func=<function get_current_time at 0x729117d1a170>)]} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


891There are 891 rows in the dataframe. The current time is not accessible through the provided context. If you want to know the current time in Python, you can run the following code:

```python
import datetime
print(datetime.datetime.now())
```

> Finished chain.


{'input': 'how many rows are there? And what is current time?',
 'output': 'There are 891 rows in the dataframe. The current time is not accessible through the provided context. If you want to know the current time in Python, you can run the following code:\n\n```python\nimport datetime\nprint(datetime.datetime.now())\n```'}